# RAPTOR: Recursive Abstractive Processing and Thematic Organization for Retrieval

Here are good resources to understand the RAPTOR:
- [rag_techniques raptor example](https://github.com/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/raptor.ipynb)
- [RAPTOR official source code](https://github.com/parthsarthi03/raptor)
- My Medium (coming soon...)

In [78]:
# Install nessary dependecy for the RAPTOR example
!pip install -U arxiv PyMuPDF # for download the document for example demonstrate
!pip install -U pydantic openai sentence-transformers transformers tiktoken scikit-learn faiss-cpu umap-learn tenacity
!pip install -U llama-index

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.5.0
    Uninstalling tenacity-8.5.0:
      Successfully uninstalled tenacity-8.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-legacy 0.9.48.post3 requires tenacity<9.0.0,>=8.2.0, but you have tenacity 9.0.0 which is incompatible.
llama-index-core 0.11.1 requires tenacity!=8.4.0,<9.0.0,>=8.2.0, but you have tenacity 9.0.0 which is incompatible.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
Using cached tenacity-8.5.0-py3-none-any.whl (28 kB)
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [79]:
import itertools
import datetime
import os
import re
import ssl
import tempfile

import typing as t
import numpy as np

from pydantic import BaseModel, Field, field_validator
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from sklearn.mixture import GaussianMixture
from tqdm.notebook import tqdm

from llama_index.core.node_parser.text import SentenceSplitter

import arxiv
import fitz
import tiktoken
import umap
import tenacity

import urllib.error
import urllib.request

from dotenv import load_dotenv

load_dotenv()

True

In [139]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # replace by your openai api key

# Config
CHUNK_SIZE = 100  # chunk size for the document
CHUNK_OVERLAP = 0  # overlap between two chunks
REDUCTION_COMPONENTS = 15  # number of components for UMAP
RANDOM_STATE = 123  # random state for GMM
MAX_LEVEL = 10  # maximum level for the clustering
MAX_TOKEN = 5000  # maximum tokens in a local cluster
MAX_CLUSTER_SIZE = 50  # maximum number of clusters
THRESHOLD = 0.1  # threshold for the clustering (depends on your document)

llm = OpenAI(api_key=OPENAI_API_KEY)
tokenizer = tiktoken.encoding_for_model("gpt-4o")

## 1. Fetch Document

First, let's install required libraries and download the document for the demonstration.
 
For the document, here choose the RAPTOR paper itself to demonstrate the RAPTOR model.

Here is the paper url:
- [RAPTOR: Recursive Abstractive Processing and Thematic Organization for Retrieval](https://arxiv.org/abs/2401.18059)

In [54]:
class ArXivPaper(BaseModel):
    title: str = Field(..., description="Title of the paper")
    text: str = Field(..., description="Text content of the paper")
    url: str = Field(..., description="URL of the paper on ArXiv")


def fetch_single_paper(url: str) -> ArXivPaper:
    """Fetches and downloads an ArXiv paper from the given URL and returns its ArXivPaper object."""
    # Ensure SSL certificate is verified
    try:
        urllib.request.urlopen(url)
    except (ssl.SSLCertVerificationError, urllib.error.URLError):
        ssl._create_default_https_context = ssl._create_unverified_context

    # Extract ArXiv ID from the URL
    arxiv_match = re.search(r"(\d{4}\.\d{4,5})(v\d+)?", url)
    if not arxiv_match:
        raise ValueError(
            f"Invalid ArXiv URL: {url}. Expected URL should contain ArXiv ID."
        )
    arxiv_id = arxiv_match.group(1)

    # Fetch paper using ArXiv ID
    client = arxiv.Client()
    search_query = arxiv.Search(id_list=[arxiv_id])
    search_result = client.results(search_query)

    try:
        paper = next(search_result)
    except StopIteration:
        raise ValueError(f"Paper not found for ArXiv ID: {arxiv_id}")

    # Download PDF and extract text content
    with tempfile.TemporaryDirectory() as temp_dir:
        pdf_path = paper.download_pdf(dirpath=temp_dir)
        doc = fitz.open(pdf_path)
        text = "".join([page.get_text() for page in doc])

    return ArXivPaper(title=paper.title, text=text, url=url)


def fetch_multiple_papers(urls: list[str]) -> list[ArXivPaper]:
    """Fetches and downloads multiple ArXiv papers from the given URLs and returns a list of ArXivPaper objects."""
    return [fetch_single_paper(url) for url in urls]

In [55]:
urls = [
    "https://arxiv.org/abs/2401.18059",
    "https://arxiv.org/abs/2307.03172",
]

papers = fetch_multiple_papers(urls)
for idx, paper in enumerate(papers):
    print(paper)

title='RAPTOR: Recursive Abstractive Processing for Tree-Organized Retrieval' text='Published as a conference paper at ICLR 2024\nRAPTOR: RECURSIVE ABSTRACTIVE PROCESSING\nFOR TREE-ORGANIZED RETRIEVAL\nParth Sarthi, Salman Abdullah, Aditi Tuli, Shubh Khanna, Anna Goldie, Christopher D. Manning\nStanford University\npsarthi@cs.stanford.edu\nABSTRACT\nRetrieval-augmented language models can better adapt to changes in world state\nand incorporate long-tail knowledge. However, most existing methods retrieve\nonly short contiguous chunks from a retrieval corpus, limiting holistic under-\nstanding of the overall document context. We introduce the novel approach of\nrecursively embedding, clustering, and summarizing chunks of text, constructing\na tree with differing levels of summarization from the bottom up. At inference\ntime, our RAPTOR model retrieves from this tree, integrating information across\nlengthy documents at different levels of abstraction. Controlled experiments show\nthat re

## 2. Split Document

> _...Construction of the RAPTOR tree begins with segmenting the retrieval corpus into short, contiguous
texts of length 100, similar to traditional retrieval augmentation techniques._

由於LLM Token limit的限制，以及太長的context會影響模型的效果。因此實務上，不論LLM是否能夠接納多長的文本，大多數的情況下都會將文本切割成較短的片段。

由於RAPTOR希望捕捉文本的深層結構，因此在Chunk的顆粒度上盡可能小一些。
作者在他的實驗中選擇了長度為100 TOKEN的Chunk，這個數字是可以根據文本的特性去調整的。
同時，作者提到如果有句子會超過100 token，那麼該句子直接放到下一個Chunk中。這樣的設計是為了保證chunk的在語意上是完整的。

我在實作中打算簡單一點，直接使用 [lammaindex SentenceSplitter](https://docs.llamaindex.ai/en/stable/api_reference/node_parsers/sentence_splitter/) 來實現這個功能。

同時為了方便之後的處理，這邊我們先定義 `Node` 的結構，之後組成樹的時候會用到。

In [56]:
class Node(BaseModel):
    text: str
    level: int = Field(default=0)
    children: set["Node"] = Field(default_factory=set)
    embedding: list[float] | None = Field(default=None)

    @field_validator("embedding", mode="before")
    @classmethod
    def convert_to_np_array(cls, v):
        if isinstance(v, list):
            return np.array(v)
        return v

In [57]:
def prepare_documents(papers: list[ArXivPaper]) -> list[Node]:
    """Prepares a list of documents from ArXivPaper."""
    splitter = SentenceSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    sentences = itertools.chain.from_iterable(
        splitter.split_text(paper.text) for paper in papers
    )
    nodes = [Node(text=sentence) for sentence in sentences]
    return nodes

In [58]:
docs = prepare_documents(papers)
docs[0]

Node(text='Published as a conference paper at ICLR 2024\nRAPTOR: RECURSIVE ABSTRACTIVE PROCESSING\nFOR TREE-ORGANIZED RETRIEVAL\nParth Sarthi, Salman Abdullah, Aditi Tuli, Shubh Khanna, Anna Goldie, Christopher D. Manning\nStanford University\npsarthi@cs.stanford.', level=0, children=set(), embedding=None)

## 3. Embed Document

> _...These texts
are then embedded using SBERT, a BERT-based encoder (multi-qa-mpnet-base-cos-v1)
(Reimers & Gurevych, 2019). The chunks and their corresponding SBERT embeddings form the
leaf nodes of our tree structure._

作者在這裡使用了SBERT來對文本進行Embedding。
這邊也採用一樣的方式。

In [59]:
embed_model = SentenceTransformer("sentence-transformers/multi-qa-mpnet-base-cos-v1")

In [60]:
def embed_documents(docs: list[Node], embeddings) -> list[Node]:
    """Embeds text for each document."""
    for doc in tqdm(docs):
        doc.embedding = embeddings.encode(doc.text)
    return docs

In [61]:
docs = embed_documents(docs, embeddings=embed_model)

  0%|          | 0/465 [00:00<?, ?it/s]

In [62]:
print(f"Shape of the embedding: {docs[0].embedding.shape}")
print(f"Example of the embedding: {docs[0].embedding[:5]}")

Shape of the embedding: (768,)
Example of the embedding: [ 0.04584955  0.07172221 -0.01249831  0.03709074 -0.03058134]


## 3. Clustering Documents

>

分群是RAPTOR的核心，其目的就是將前面文本切割成的小片段進行聚合，這樣可以更好的捕捉文本的深層結構。接著透過遞迴的方式，將整個文本的結構由最底層的片段逐步的聚合到最上層，最後產生一個樹狀的結構，這個結構就是RAPTOR的核心概念。

作者在這裡使用了GMM來進行分群。 

在實作中，大致上分成 **local clustering** 和 **global clustering** 兩個部分。


In [136]:
class ClusterResult(t.NamedTuple):
    clusters_array: np.ndarray
    n_clusters: int


class RAPTORClustering:

    def __init__(
        self,
        reduction_components: int,
        threshold: float,
        random_state: int,
        max_cluster_size: int,
    ):
        self.reduction_components = reduction_components
        self.threshold = threshold
        self.random_state = random_state
        self.max_cluster_size = max_cluster_size

    def __call__(self, embeddings: np.ndarray):
        embeddings = embeddings.copy()

        doc_size, _ = embeddings.shape

        total_cluster_count = 0
        total_clusters_labels = [
            np.array([], dtype=int) for _ in range(doc_size)
        ]  # [(doc_size, 0)]

        # * Global clustering
        # Use large n_neighbors for global clustering to capture the global structure.
        # In the author's implementation, they use n_neighbors = sqrt(embeddings_size - 1)
        global_n_neighbors = max(int((doc_size - 1) ** 0.5), 2)
        global_cluster_labels, global_n_clusters = self.cluster(
            embeddings=embeddings,
            n_components=self.reduction_components,
            n_neighbors=global_n_neighbors,
        )

        print(f"Number of global clusters: {global_n_clusters}")

        # * Local clustering
        for global_cluster_idx in tqdm(
            range(global_n_clusters), desc="Local Clustering..."
        ):
            # Extract embeddings for each global cluster
            embeddings_in_global_cluster = embeddings[
                np.array([global_cluster_idx in gc for gc in global_cluster_labels])
            ]

            # Handle the empty cluster
            # Sometimes, GMM may not find any cluster for the given threshold
            if len(embeddings_in_global_cluster) == 0:
                continue

            # Determine if local clustering is needed
            # If the number of embeddings in this global cluster is too small (<= reduction components + 1),
            # further clustering is unnecessary. Assign all embeddings to a single local cluster.
            if len(embeddings_in_global_cluster) <= self.reduction_components + 1:
                local_cluster_labels = [
                    np.array([0]) for _ in embeddings_in_global_cluster
                ]
                n_local_clusters = 1
            else:
                # * Perform local clustering
                # Use small n_neighbors for local clustering to capture the local structure.
                # In the author's implementation, they use n_neighbors = 10
                local_cluster_labels, n_local_clusters = self.cluster(
                    embeddings=embeddings_in_global_cluster,
                    n_components=self.reduction_components,
                    n_neighbors=10,
                )

            # * Update the total_clusters_label with local cluster assignments
            for local_cluster_idx in range(n_local_clusters):
                self.update_cluster_labels(
                    embeddings=embeddings,
                    embeddings_in_global_cluster=embeddings_in_global_cluster,
                    local_cluster_label=local_cluster_labels,
                    total_clusters_label=total_clusters_labels,
                    local_cluster_idx=local_cluster_idx,
                    total_cluster_count=total_cluster_count,
                )

            total_cluster_count += n_local_clusters

        print(f"Total number of clusters: {total_cluster_count}")

        return total_clusters_labels

    # @tenacity.retry(stop=(tenacity.stop_after_delay(20) | tenacity.stop_after_attempt(5)))
    def cluster(
        self, embeddings: np.ndarray, n_components: int, n_neighbors: int = 10
    ) -> ClusterResult:
        # Reduce the dimensionality by UMAP
        reduced_embeddings = umap.UMAP(
            n_neighbors=n_neighbors,
            n_components=n_components,
            metric="cosine",
        ).fit_transform(embeddings)

        # Find the optimal number of clusters
        n_clusters = self.find_cluster_count(reduced_embeddings)

        # Perform clustering
        gmm = GaussianMixture(n_components=n_clusters, random_state=self.random_state)
        gmm.fit(reduced_embeddings)

        # Assign the cluster to each embedding
        probs = gmm.predict_proba(reduced_embeddings)

        # Given threshold, assign the cluster
        clusters_array = np.array(
            [np.where(prob >= self.threshold)[0] for prob in probs]
        )  # (doc_size, 1)

        return ClusterResult(clusters_array=clusters_array, n_clusters=n_clusters)

    def find_cluster_count(self, embeddings: np.ndarray) -> int:
        doc_size, _ = embeddings.shape
        # ? Determine the maximum number of clusters, ensuring it doesn't exceed the document count.
        # ? The number of clusters should not exceed the number of documents because:
        # ? 1. Each document could potentially be its own cluster, but you can't have more clusters than documents.
        # ? 2. Clustering aims to group similar documents, so having more clusters than documents is not meaningful.
        clusters_size = min(self.max_cluster_size, doc_size)
        n_clusters = np.arange(1, clusters_size)
        bics = []
        for n in n_clusters:
            gm = GaussianMixture(n_components=n, random_state=self.random_state)
            gm.fit(embeddings)
            bics.append(gm.bic(embeddings))
        return n_clusters[np.argmin(bics)]

    def update_cluster_labels(
        self,
        embeddings: np.ndarray,
        embeddings_in_global_cluster: np.ndarray,
        local_cluster_label: np.ndarray,
        total_clusters_label: list[np.ndarray],
        local_cluster_idx: int,
        total_cluster_count: int,
    ) -> None:
        """Update the total_clusters_label with the new cluster index."""

        # Identify which embeddings belong to the current local cluster
        is_in_local_cluster = np.array(
            [local_cluster_idx in label for label in local_cluster_label]
        )
        embeddings_in_local_cluster = embeddings_in_global_cluster[is_in_local_cluster]

        # Find the indices of these embeddings in the original embeddings array
        expanded_embeddings = embeddings_in_local_cluster[:, np.newaxis]
        comparison_matrix = embeddings == expanded_embeddings
        matches = comparison_matrix.all(axis=-1)
        indices = np.where(np.any(matches, axis=0))[0]

        # Update the total_clusters_label with the new cluster index
        new_cluster_index = local_cluster_idx + total_cluster_count
        for idx in indices:
            current_label = total_clusters_label[idx]
            total_clusters_label[idx] = np.append(current_label, new_cluster_index)

clustering = RAPTORClustering(
    reduction_components=40,
    threshold=THRESHOLD,
    random_state=RANDOM_STATE,
    max_cluster_size=MAX_CLUSTER_SIZE,
)
embeddings = np.array([doc.embedding for doc in docs])
cluster_label = clustering(embeddings)

Number of global clusters: 3


Local Clustering...:   0%|          | 0/3 [00:00<?, ?it/s]

Total number of clusters: 9


In [142]:
def run_clustering(
    nodes: list[Node],
    reduction_components: int = REDUCTION_COMPONENTS,
    threshold: float = THRESHOLD,
    random_state: int = RANDOM_STATE,
    max_cluster_size: int = MAX_CLUSTER_SIZE,
    max_token: int = MAX_TOKEN,
) -> list[Node]:
    """Runs the RAPTOR clustering algorithm on the given nodes."""
    model = RAPTORClustering(
        reduction_components=reduction_components,
        threshold=threshold,
        random_state=random_state,
        max_cluster_size=max_cluster_size,
    )

    # * Perform clustering
    embeddings = np.array([node.embedding for node in nodes])
    cluster_labels = model(embeddings)
    n_cluster_labels = np.unique(np.concatenate(cluster_labels))

    result = []
    for label in n_cluster_labels:
        # Get the indices of the nodes that belong to this cluster
        indices = [i for i, cluster in enumerate(cluster_labels) if label in cluster]

        # Add the corresponding nodes to the node_clusters list
        cluster_nodes = [nodes[i] for i in indices]

        # Base case: if the cluster only has one node, do not attempt to recluster it
        if len(cluster_nodes) == 1:
            result.append(cluster_nodes)
            continue

        # Calculate the total length of the text in the nodes
        total_length = sum([len(tokenizer.encode(node.text)) for node in cluster_nodes])

        # If the total length exceeds the maximum allowed length, re-cluster this cluster
        if total_length > max_token:
            result.extend(run_clustering(nodes=cluster_nodes))
        else:
            result.append(cluster_nodes)

    return result

result = run_clustering(nodes=docs, reduction_components=40, max_cluster_size=30)

Number of global clusters: 4


Local Clustering...:   0%|          | 0/4 [00:00<?, ?it/s]

Total number of clusters: 8
Number of global clusters: 6


Local Clustering...:   0%|          | 0/6 [00:00<?, ?it/s]

Total number of clusters: 11
Number of global clusters: 6


...

Total number of clusters: 7
Number of global clusters: 4


Local Clustering...:   0%|          | 0/4 [00:00<?, ?it/s]

Total number of clusters: 16
Number of global clusters: 6


Local Clustering...:   0%|          | 0/6 [00:00<?, ?it/s]

Total number of clusters: 8


In [156]:
size = sum([len(cluster) for cluster in result])
assert size == len(docs), f"Expected {len(docs)} nodes, but got {size} nodes."

NameError: name 'node' is not defined

In [ ]:
clustering = RAPTORClustering()
embeddings = np.array([doc.embedding for doc in docs])
clusters_array, n_clusters = clustering.cluster(embeddings)

print(f"Number of documents: {len(clusters_array)}")

In [ ]:
clustering.get_num_clusters(clusters_array)

In [ ]:
len(set([cluster[0] for cluster in clusters_array]))

## 4. Dimension Reduction (UMAP)

> _...The high dimensionality of vector embeddings presents a challenge for traditional GMMs, as dis-
tance metrics may behave poorly when used to measure similarity in high-dimensional spaces (Ag-
garwal et al., 2001)._

Embedding之後，就可以準備分群。

不過在分群之前，作者提到了SBERT的Embedding是高維的 (768)，這對於傳統的GMM模型來說是一個挑戰。

因此這邊此用UMAP來對Embedding進行降維，作者降到10維。

> _Our algorithm varies `n_neighbors` to create a hierar-
chical clustering structure: it first identifies **global clusters** and then performs **local clustering** within
these global clusters. This two-step clustering process captures a broad spectrum of relationships
among the text data, from broad themes to specific details._

作者在這裡提到了他們的算法是通過調整`n_neighbors`來創建一個層次化的聚類結構，有兩個步驟：
1. Global Clustering： 這個步驟是為了找到全局的Cluster。
2. Local Clustering： 在全局的Cluster中進行局部的Clustering。這樣的步驟可以捕捉到從廣泛的主題到具體細節的關係。

> _Should a local cluster’s combined context ever exceed the summarization model’s token threshold, our algorithm recursively applies clustering within the cluster, ensuring that the context remains within the token threshold._

為什麼要分成global和local呢？作者在這裡提到了這樣的設計是為了捕捉到文本的深層結構。
還有一個重要的原因是，由於token limitation，當global分群的文本太長時，沒辦法直接將其所有文本拿去summary。
因此，作者在這裡提到了如果local cluster的文本超過了summarization model的token threshold，那麼他們會在這個cluster中再次應用clustering，直到文本的長度在summarization model的token threshold之內。

In [ ]:
import umap

# First, we do global clustering
n_neighbors = int(
    (len(nodes[0].embedding) - 1) ** 0.5
)  # large n_neighbor to capture the global structure
umap_model = umap.UMAP(
    n_components=10,
    n_neighbors=n_neighbors,
    metric="cosine",
)

embeddings = np.array([node.embedding for node in nodes])
reduced_embeddings = umap_model.fit_transform(embeddings)
print(reduced_embeddings.shape)  # (sample_size, 10)
print(reduced_embeddings[0])

## 5. Clustering (GMM)

> _Clustering plays a key role in building the RAPTOR tree, organizing text
segments into cohesive groups. This step groups related content together, which helps the subsequent retrieval process....One of the unique aspects of our clustering approach is the use of soft clustering, where nodes can
belong to multiple clusters without requiring a fixed number of clusters. This flexibility is essential because individual text segments often contain information relevant to various topics, thereby
warranting their inclusion in multiple summaries. Our clustering algorithm is based on Gaussian Mixture Models (GMMs)..._

準備後降為後的Embedding進行分群了，這是RAPTOR最核心的一個步驟。

作者在這裡使用了GMM來進行分群，並且使用了soft clustering的方式，這樣可以讓一個文本屬於多個Cluster。這樣的方法，可以讓RAPTOR更好的捕捉文本的深層結構，並將相關的文本放到一起。




In [ ]:
from sklearn.mixture import GaussianMixture


def find_optimal_clusters(
    embeddings: np.ndarray, *, max_clusters: int, random_state: int
) -> int:
    max_clusters = min(max_clusters, len(embeddings))
    n_clusters = np.arange(1, max_clusters)
    bics = []
    for n in tqdm(n_clusters, desc="Finding optimal number of clusters"):
        gm = GaussianMixture(n_components=n, random_state=random_state)
        gm.fit(embeddings)
        bics.append(gm.bic(reduced_embeddings))
    return n_clusters[np.argmin(bics)]


n_global_clusters = find_optimal_clusters(
    reduced_embeddings, max_clusters=50, random_state=random_state
)
print(f"Optimal number of clusters: {n_global_clusters}")

找到 optimal number of clusters 之後就可以開始分群了。

In [ ]:
gmm = GaussianMixture(n_components=n_global_clusters, warm_start=True, verbose=1)
gmm.fit(reduced_embeddings)
probs = gmm.predict_proba(reduced_embeddings)

In [ ]:
probs

In [ ]:
threshold = 0.1
labels = [np.where(prob > threshold)[0] for prob in probs]
labels

In [ ]:
all_local_clusters = [np.array([]) for _ in range(len(embeddings))]
total_clusters = 0

for global_idx in range(n_global_clusters):
    global_clustering_idx = np.array([global_idx in gc for gc in labels])
    global_cluster_embeddings_ = embeddings[global_clustering_idx]
    print(f"Nodes in Global Cluster {global_idx}: {len(global_cluster_embeddings_)}")
    ####################
    # Local clustering #
    ####################

    # sometimes, GMM may generate cluster with empty data points.
    # In this case, we skip this local clustering.
    if len(global_cluster_embeddings_) == 0:
        continue

    # Get the number of local clusters

    # If the cluster size is ≤ dim + 1, meaningful clustering isn't possible due to insufficient points.
    if len(global_cluster_embeddings_) <= n_components + 1:
        local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
        n_local_clusters = 1
    # find the optimal number of local clusters
    else:
        local_cluster_embeddings_ = umap.UMAP(
            n_components=n_components,
            n_neighbors=10,  # use small n_neighbors to capture the local structure
            metric="cosine",
        ).fit_transform(global_cluster_embeddings_)
        n_local_clusters = find_optimal_clusters(
            local_cluster_embeddings_, max_clusters=50, random_state=random_state
        )
        gmm = GaussianMixture(n_components=n_local_clusters, random_state=random_state)
        gmm.fit(local_cluster_embeddings_)
        probs = gmm.predict_proba(local_cluster_embeddings_)
        local_clusters = [np.where(prob > threshold)[0] for prob in probs]

    for local_idx in range(n_local_clusters):
        local_cluster_embeddings_ = global_cluster_embeddings_[
            np.array([local_idx in lc for lc in local_clusters])
        ]
        indices = np.where((embeddings == local_cluster_embeddings_[:, None]).all(-1))[
            1
        ]
        for idx in indices:
            all_local_clusters[idx] = np.append(
                all_local_clusters[idx], local_idx + total_clusters
            )

    total_clusters += n_local_clusters

In [ ]:
print(all_local_clusters)

In [ ]:
print(total_clusters)

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")
labels = np.unique(np.concatenate(all_local_clusters))
node_clusters = []

for label in labels:
    indices = [i for i, cluster in enumerate(all_local_clusters) if label in cluster]
    print(f"Cluster {label}: {indices}")

    cluster_nodes = [nodes[i] for i in indices]

    # Base case: if the cluster only has one node, do not attempt to recluster it
    if len(cluster_nodes) == 1:
        node_clusters.append(cluster_nodes)
        continue

    # Calculate the total length of the text in the nodes
    total_length = sum([len(tokenizer.encode(node.text)) for node in cluster_nodes])
    print(f"Total length: {total_length}")

    # If the total length exceeds the maximum allowed length, recluster this cluster
    if total_length > max_length_in_cluster:
        ...

In [ ]:
class RAPTORClustering:

    def __init__(self, n_components: int = 10, max_clusters: int = 50):
        self.n_components = n_components
        self.max_clusters = max_clusters

    def find_optimal_clusters(self, embeddings: np.ndarray) -> int:
        max_clusters = min(max_clusters, len(embeddings))
        n_clusters = np.arange(1, max_clusters)
        bics = []
        for n in tqdm(n_clusters, desc="Finding optimal number of clusters"):
            gm = GaussianMixture(n_components=n)
            gm.fit(embeddings)
            bics.append(gm.bic(reduced_embeddings))
        return n_clusters[np.argmin(bics)]

    def umap(
        self,
        embeddings: np.ndarray,
        *,
        n_components: int,
        n_neighbors: int | None = None,
        metric: str = "cosine",
    ) -> np.ndarray:
        return umap.UMAP(
            n_neighbors=n_neighbors, n_components=n_components, metric=metric
        ).fit_transform(embeddings)

    def _cluster_embeddings(self): ...

    def cluster(self, embeddings: np.ndarray, *, threshold: float = 0.1):

        # Global clustering
        n_neighbors = int(
            (len(embeddings[0]) - 1) ** 0.5
        )  # use large n_neighbor to capture the global structure
        reduced_embeddings = self.umap(
            embeddings, n_components=self.n_components, n_neighbors=n_neighbors
        )

        # Find optimal number of clusters for global clustering
        n_global_clusters = self.find_optimal_clusters(reduced_embeddings)

        # Cluster embeddings based no optimal number of clusters
        gmm = GaussianMixture(n_components=n_global_clusters)
        gmm.fit(embeddings)
        probs = gmm.predict_proba(embeddings)

        # Get global clusters based on threshold
        global_clusters = [np.where(prob > threshold)[0] for prob in probs]

        print(f"Number of Global Clusters: {len(n_global_clusters)}")

        for idx, cluster in enumerate(n_global_clusters):
            global_cluster_embeddings_ = embeddings[
                np.array([idx in gc for gc in global_clusters])
            ]
            print(f"Nodes in Global Cluster {idx}: {len(global_cluster_embeddings_)}")

            ####################
            # Local clustering #
            ####################

            # sometimes, GMM may generate cluster with empty data points.
            # In this case, we skip this local clustering.
            if len(global_cluster_embeddings_) == 0:
                continue

            # Get the number of local clusters

            # If the cluster size is ≤ dim + 1, meaningful clustering isn't possible due to insufficient points.
            if len(global_cluster_embeddings_) <= n_components + 1:
                local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
                n_local_clusters = 1
            # find the optimal number of local clusters
            else:
                local_cluster_embeddings_ = umap.UMAP(
                    n_components=n_components,
                    n_neighbors=10,  # use small n_neighbors to capture the local structure
                    metric="cosine",
                ).fit_transform(global_cluster_embeddings_)

In [ ]:
LLM_AS_JUDGE_TEMPLATE = """\
As a cybersecurity expert, evaluate the alert description and risk level based on the attack scenario.
Follow these steps:
1. Develop a scoring guideline specific to the attack scenario, with a 0 to 10 scale. Define how points are awarded based on clear criteria.
2. Assign a score from 0 to 10, and explain how well the alert meets the criteria.

ATTACK SCENARIO: ```{attack_scenario}```

ALERT DESCRIPTION: ```{alert_description}```
ALERT RISK LEVEL: ```{alert_risk_level}```
"""

llm_judge_response = llm.request_chat(
    content=LLM_AS_JUDGE_TEMPLATE.format(
        attack_scenario=attack_scenario,
        alert_description=alert["description"],
        alert_risk_level=alert["riskLevel"],
    )
)

print(llm_judge_response)